Make sure Python and Jupyter are set up locally first (and if necessary relaunch VSCode). There is a `shell.nix` that does this for you if you have Nix installed, but it won't work with some of the Spring AI library dependencies (the ones that have native code extensions), so it's better to use a virtual env created from the global Python.

```bash
$ python3 -m venv .venv
$ . .venv/bin/activate
$ pip install jupyter ipykernel notebook
```

Then, select the Python interpreter in the top right corner of the VSCode editor - it should be `java (Rapaio/j!)`. If that is not available, install it with `jbang install-kernel@jupyter-java rapaio`. Make sure to get the latest version of `jbang` and also edit the `kernel.json` file in `~/.local/share/jupyter/kernels` to upgrade to version 2.0.0 and Java 22.

```json
{
  "argv" : [
    "/home/dsyer/.sdkman/candidates/jbang/current/bin/jbang",
    "--java",
    "22",
...
    "io.github.padreati:rapaio-jupyter-kernel:2.0.0@fatjar",
    "{connection_file}"
  ],
...
}
```

In [1]:
%classpath target/classes/

Add /home/dsyer/dev/scratch/enhance-llm/target/classes to classpath


In [5]:
%%jars
target/lib

Found 130 jar files.
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-datatype-jdk8-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/typetools-0.6.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/spring-boot-starter-logging-3.3.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-core-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/logback-classic-1.5.6.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/spring-ai-core-1.0.0-SNAPSHOT.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/commons-pool2-2.12.0.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/json-20231013.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/hamcrest-core-2.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-databind-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jtokkit-1.1.0.jar to

In [3]:
import org.springframework.boot.logging.*;
// Make sure Spring Boot is using the Java Logging System not Logback
System.setProperty(LoggingSystem.SYSTEM_PROPERTY, "org.springframework.boot.logging.java.JavaLoggingSystem");

Run the vector store in the background:

```bash
$ docker run -it --rm --name chroma -p 8000:8000 ghcr.io/chroma-core/chroma:0.4.15
```

and then the app will be able to connect to it when it starts up.

Models prepared in onxx format:

```bash
$ pip install optimum onnx onnxruntime
$ optimum-cli export onnx --model sentence-transformers/multi-qa-MiniLM-L6-cos-v1 onnx-output
```

In [10]:
%%bash
ls -l onnx-output

total 102052
-rw-r--r-- 1 dsyer dsyer      697 Aug  5 09:55 config.json
-rw-r--r-- 1 dsyer dsyer 13085339 Aug  5 09:59 databricks-dolly-15k.jsonl
-rw-r--r-- 1 dsyer dsyer 90447733 Aug  5 09:55 model.onnx
-rw-r--r-- 1 dsyer dsyer      695 Aug  5 09:55 special_tokens_map.json
-rw-r--r-- 1 dsyer dsyer   711661 Aug  5 09:55 tokenizer.json
-rw-r--r-- 1 dsyer dsyer     1433 Aug  5 09:55 tokenizer_config.json
-rw-r--r-- 1 dsyer dsyer   231508 Aug  5 09:55 vocab.txt


```bash
$ ollama serve
$ ollama pull mistral
```

In [6]:
import org.springframework.boot.SpringApplication;
import com.example.DemoApplication;
var app = SpringApplication.run(DemoApplication.class, new String[] {});

[Engine-thread-1] INFO org.springframework.boot.devtools.env.DevToolsPropertyDefaultsPostProcessor - For additional web related logging consider setting the 'logging.level.web' property to 'DEBUG'



  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/

 :: Spring Boot ::                (v3.3.2)



[Engine-thread-1] INFO org.springframework.boot.SpringApplication - Starting application using Java 22.0.2 with PID 2959169 (started by dsyer in /home/dsyer/dev/scratch/enhance-llm)
[Engine-thread-1] INFO org.springframework.boot.SpringApplication - No active profile set, falling back to 1 default profile: "default"
[Engine-thread-1] INFO org.springframework.boot.web.embedded.tomcat.TomcatWebServer - Tomcat initialized with port 8080 (http)
[Engine-thread-1] INFO org.springframework.boot.web.servlet.context.ServletWebServerApplicationContext - Root WebApplicationContext: initialization completed in 266 ms
[Engine-thread-1] INFO org.springframework.ai.transformers.ResourceCacheService - Create cache root directory: /tmp/spring-ai-onnx-generative
[Engine-thread-1] INFO org.springframework.ai.transformers.ResourceCacheService - The URL [file:/home/dsyer/dev/scratch/enhance-llm/onnx-output/tokenizer.json] resource with URI schema [file] is excluded from caching
[Engine-thread-1] INFO ai.dj

In [7]:
import org.springframework.ai.embedding.EmbeddingModel;
var model = app.getBean(EmbeddingModel.class);
model.embed("Hello World");

[Engine-thread-1] INFO ai.djl.pytorch.jni.LibUtils - Downloading https://publish.djl.ai/pytorch/2.2.2/cpu/linux-x86_64/native/lib/libc10.so.gz ...
[Engine-thread-1] INFO ai.djl.pytorch.jni.LibUtils - Downloading https://publish.djl.ai/pytorch/2.2.2/cpu/linux-x86_64/native/lib/libtorch_cpu.so.gz ...
[Engine-thread-1] INFO ai.djl.pytorch.jni.LibUtils - Downloading https://publish.djl.ai/pytorch/2.2.2/cpu/linux-x86_64/native/lib/libgomp-98b21ff3.so.1.gz ...
[Engine-thread-1] INFO ai.djl.pytorch.jni.LibUtils - Downloading https://publish.djl.ai/pytorch/2.2.2/cpu/linux-x86_64/native/lib/libtorch.so.gz ...
[Engine-thread-1] INFO ai.djl.pytorch.jni.LibUtils - Downloading jni https://publish.djl.ai/pytorch/2.2.2/jnilib/0.28.0/linux-x86_64/cpu/libdjl_torch.so to cache ...
[Engine-thread-1] INFO ai.djl.pytorch.engine.PtEngine - PyTorch graph executor optimizer is enabled, this may impact your inference latency and throughput. See: https://docs.djl.ai/docs/development/inference_performance_optimi

[0.04318665340542793, 0.30097007751464844, 0.26396337151527405, 0.04484155774116516, 0.003967605531215668, -0.42618435621261597, 0.5495190620422363, -0.33905696868896484, -0.38261377811431885, 0.10058178752660751, 0.15808290243148804, -0.623440146446228, 0.08899436891078949, 0.025170493870973587, 0.5159124135971069, 0.12556859850883484, 0.11611807346343994, -0.36689549684524536, -0.7174185514450073, -0.2329343855381012, -0.14610600471496582, -0.008427665568888187, 0.44609546661376953, 0.25121358036994934, -0.25413763523101807, 0.022199597209692, 0.04044394940137863, 0.3187928795814514, 0.4143705666065216, -0.022700220346450806, -0.3824435770511627, -0.07699090242385864, 0.8247401714324951, -0.08617451041936874, -0.10628770291805267, 0.2115754783153534, 0.07268550992012024, -0.528840959072113, -0.2989896237850189, 0.07533113658428192, -0.0048542022705078125, -0.15591315925121307, 0.1626037359237671, 0.5093824863433838, -0.08490810543298721, -0.0011676587164402008, -0.2136777937412262, 0

In [8]:
model.embed("Hello London");

[0.4970059394836426, -0.12482398748397827, 0.8155074715614319, -0.2911941409111023, -0.4481144845485687, -0.11870981752872467, 0.71401047706604, -0.838150680065155, -0.7358101010322571, -0.044329360127449036, 0.18298369646072388, -0.3730786442756653, -0.07361304014921188, -0.2382127046585083, 0.4210531711578369, 0.1900179088115692, 0.22915011644363403, -0.5986695289611816, -0.6124140024185181, -0.43450912833213806, 0.05107399821281433, -0.07223984599113464, 0.25163063406944275, 0.49134063720703125, -0.37957972288131714, -0.04222181439399719, 0.16610795259475708, 0.5360963344573975, 0.46843382716178894, -0.05512235313653946, -0.3537119925022125, -0.48311400413513184, 0.6804139018058777, -0.07597935199737549, 0.32974013686180115, 0.5870456695556641, 0.3543405532836914, -0.7406903505325317, -0.34177297353744507, -0.06377626955509186, -0.22146177291870117, -0.5120139718055725, 0.23788021504878998, 0.27635657787323, 0.1982792615890503, 0.07828925549983978, 0.1046694964170456, -0.10592179000

In [9]:
import com.fasterxml.jackson.databind.ObjectMapper;
var mapper = app.getBean(ObjectMapper.class)

In [10]:
var reader = mapper.readerFor(Map.class)

In [11]:
reader.readValues(new File("./data/lines.jsonl")).forEachRemaining(line -> System.out.println(((Map)line).get("instruction")));

How to make a cup of tea?
What is the capital of France?
What is the capital of Germany?
What is the capital of Italy?
What is the capital of Spain?
What is the capital of Portugal?
What is the capital of Greece?
What is the capital of Turkey?
What is the capital of Egypt?
What is the capital of South Africa?
What is the capital of Nigeria?
What is the capital of Kenya?
What is the capital of India?
What is the capital of China?
What is the capital of Japan?
What is the capital of Australia?
What is the capital of New Zealand?
What is the capital of Canada?
What is the capital of the United States?
What is the capital of Brazil?
What is the capital of Argentina?
What is the capital of Chile?
What is the capital of Peru?


In [11]:
import org.springframework.ai.document.Document;
import org.springframework.ai.vectorstore.VectorStore;

var store = app.getBean(VectorStore.class);

The dataset is stored in huggingface in `git-lfs` format, so you need to install `git-lfs` and then run `git lfs install && git lfs checkout` in the repo to download the data. Or download it manually from the [Hugging Face website](https://huggingface.co/datasets/databricks-dolly-15k). Or use the HugingFace CLI:

```bash
$ huggingface-cli download --local-dir onnx-output --repo-type dataset databricks/databricks-dolly-15k databricks-dolly-15k.jsonl
```

In [14]:
import org.springframework.util.StringUtils;
import org.springframework.ai.transformer.splitter.TokenTextSplitter;
List<Document> list = new ArrayList<>();
TokenTextSplitter splitter = new TokenTextSplitter(200, 50, 5, 400, true);
reader.readValues(new File("./onnx-output/databricks-dolly-15k.jsonl")).forEachRemaining(line -> {
	Map map = (Map) line;
	if (!"closed_qa".equals(map.get("category"))) {
		return;
	}
	String context = ((String) map.get("context")).trim();
	if (StringUtils.hasText(context)) {
		Document doc = new Document(context);
		List<Document> splitDocs = splitter.split(doc);
		list.addAll(splitDocs);
	}
});
store.add(list);

[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 3 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-2] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 4 chunks.
[Engine-thread-2] INFO org.springframework.ai.tr

In [15]:
store.similaritySearch("When was Tomoaki Komorida born?")

[Document{id='e014ec9d-49da-4a8e-a0a0-3ec5a1d77394', metadata={distance=0.4065361}, content='Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer.', media=[]}, Document{id='1374e307-fea9-464d-b399-ac43fcc2caf8', metadata={distance=0.5872118}, content='Woods was born on December 30, 1975, in Cypress, California, to Earl and Kultida "Tida" Woo

In [16]:
import org.springframework.ai.chat.model.ChatModel;
var chat = app.getBean(ChatModel.class);

In [17]:
chat.call("When was Tomoaki Komorida born?")

 I don't have real-time data or the ability to browse the internet. As of my last update, Tomoaki Komorida is a professional Japanese shogi player who was born on June 26, 1985. However, I recommend verifying this information from a reliable source to ensure its accuracy.

In [19]:
import org.springframework.ai.chat.client.ChatClient;
import org.springframework.ai.chat.client.advisor.QuestionAnswerAdvisor;
import org.springframework.ai.vectorstore.SearchRequest;
var builder = app.getBean(ChatClient.Builder.class);
var response = builder.build().prompt()
.advisors(new QuestionAnswerAdvisor(store, SearchRequest.defaults()))
.user("When was Tomoaki Komorida born?")
.call()
.chatResponse();

ChatResponse [metadata={ id: , usage: org.springframework.ai.chat.metadata.EmptyUsage@53f82e7c, rateLimit: org.springframework.ai.chat.metadata.EmptyRateLimit@13b69000 }, generations=[Generation[assistantMessage=AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent= Tomoaki Komorida was born on July 10, 1981., metadata={messageType=ASSISTANT}], chatGenerationMetadata=ChatGenerationMetadata{finishReason=stop,contentFilterMetadata=null}]]]

In [28]:
response.content()

 Tomoaki Komorida was born on July 10, 1981.

In [30]:
splitter.split(new Document("Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia and joined Persela Lamongan. In July 2010, he returned to Japan and joined the J2 club Giravanz Kitakyushu. He played often as a defensive midfielder and center back until 2012 when he retired."))

[Engine-thread-5] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.


[Document{id='dc6b45a0-5014-4231-8b1a-654e1dd80999', metadata={}, content='Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer.', media=[]}, Document{id='38c4969b-5cb7-49f3-a360-3894caf38bc6', metadata={}, content='In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played